##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Tutorial: um pipeline TFX simples usando o dataset Penguin

***Um breve tutorial para executar um pipeline TFX simples.***

Observação: recomendamos executar este tutorial em um notebook Colab, sem necessidade de configuração! Basta clicar em “Executar no Google Colab”.

<div class="devsite-table-wrapper"><table class="tfo-notebook-buttons" align="left">
<td>     <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/tfx/penguin_simple"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
<td><a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/penguin_simple.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a></td>
<td><a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tfx/tutorials/tfx/penguin_simple.ipynb"><img width="32px" src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a></td>
<td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tfx/tutorials/tfx/penguin_simple.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table></div>

Neste tutorial baseado em notebook, criaremos e executaremos um pipeline TFX para um modelo de classificação simples. O pipeline consistirá em três componentes essenciais do TFX: ExampleGen, Trainer e Pusher. O pipeline inclui um fluxo de trabalho de ML mínimo, como importação de dados, treinamento de um modelo e exportação do modelo treinado.

Veja [Introdução aos pipelines do TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para saber mais sobre vários conceitos do TFX.

## Configuração

Primeiro precisamos instalar o pacote TFX Python e baixar o dataset que usaremos para nosso modelo.

### Atualize o Pip

Para evitar a atualização do Pip num sistema ao executar localmente, garanta que estamos executando no Colab. Os sistemas locais podem, claro, ser atualizados separadamente.

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

### Instale o TFX


In [ ]:
!pip install -U tfx

### Desinstale o shapely

TODO(b/263441833) Esta é uma solução temporária para evitar um ImportError. Em última análise, isto deverá ser resolvido com suporte a uma versão mais recente do Bigquery, em vez de desinstalar outras dependências extras.

In [ ]:
!pip uninstall shapely -y

### Você reiniciou o runtime?

Se você estiver usando o Google Colab, na primeira vez que executar a célula acima, você deve reiniciar o runtime clicando no botão "RESTART RUNTIME" acima ou usando o menu "Runtime &gt; Restart runtime ...". Isso é necessário devido à maneira como o Colab carrega os pacotes.

Verifique as versões do TensorFlow e TFX.

In [ ]:
import tensorflow as tf
print('TensorFlow version: {}'.format(tf.__version__))
from tfx import v1 as tfx
print('TFX version: {}'.format(tfx.__version__))

### Configuração de variáveis

Existem algumas variáveis ​​usadas para definir um pipeline. Você pode personalizar essas variáveis ​​como desejar. Por padrão, toda a saída do pipeline será gerada no diretório atual.

In [ ]:
import os

PIPELINE_NAME = "penguin-simple"

# Output directory to store artifacts generated from the pipeline.
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
# Path to a SQLite DB file to use as an MLMD storage.
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
# Output directory where created models from the pipeline will be exported.
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)

from absl import logging
logging.set_verbosity(logging.INFO)  # Set default logging level.

### Preparação dos dados de exemplo

Faremos download do dataset de exemplo para uso no nosso pipeline TFX. O dataset que estamos usando é o [dataset Palmer Penguins](https://allisonhorst.github.io/palmerpenguins/articles/intro.html) , que também é usado em outros [exemplos com TFX](https://github.com/tensorflow/tfx/tree/master/tfx/examples/penguin).

Há quatro características numéricas neste dataset:

- culmen_length_mm
- culmen_depth_mm
- flipper_length_mm
- body_mass_g

Todas as características já foram normalizadas para ter um intervalo [0,1]. Construiremos um modelo de classificação que prevê as `species` de pinguins.

Como o TFX ExampleGen lê entradas de um diretório, precisamos criar um diretório e copiar o dataset para ele.

In [ ]:
import urllib.request
import tempfile

DATA_ROOT = tempfile.mkdtemp(prefix='tfx-data')  # Create a temporary directory.
_data_url = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/penguin/data/labelled/penguins_processed.csv'
_data_filepath = os.path.join(DATA_ROOT, "data.csv")
urllib.request.urlretrieve(_data_url, _data_filepath)

Dê uma olhada rápida no arquivo CSV.

In [ ]:
!head {_data_filepath}

Você deve conseguir ver cinco valores. `species` é 0, 1 ou 2, e todas as outras características devem ter valores entre 0 e 1.

## Crie um pipeline

Os pipelines TFX são definidos usando APIs Python. Definiremos um pipeline que consiste nos seguintes três componentes.

- CsvExampleGen: lê arquivos de dados e os converte para o formato interno do TFX para processamento posterior. Existem diversos [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) para vários formatos. Neste tutorial, usaremos CsvExampleGen, que recebe como entrada um arquivo CSV.
- Trainer: treina um modelo de ML. O [componente Trainer](https://www.tensorflow.org/tfx/guide/trainer) requer um código de definição de modelo dos usuários. Você pode usar APIs do TensorFlow para especificar como treinar um modelo e salvá-lo em formato *save_model*.
- Pusher: copia o modelo treinado fora do pipeline do TFX. O [componente Pusher](https://www.tensorflow.org/tfx/guide/pusher) pode ser pensado como um processo de implantação do modelo de ML treinado.

Antes de realmente definir o pipeline, precisamos primeiro escrever um código de modelo para o componente Trainer.

### Escreva o código para treinamento do modelo

Criaremos um modelo DNN simples para classificação usando a API TensorFlow Keras. Este código de treinamento do modelo será salvo num arquivo separado.

Neste tutorial usaremos o [Generic Trainer](https://www.tensorflow.org/tfx/guide/trainer#generic_trainer) do TFX que suporta modelos baseados em Keras. Você precisa escrever um arquivo Python contendo a função `run_fn`, que é o ponto de entrada para o componente `Trainer`.

In [ ]:
_trainer_module_file = 'penguin_trainer.py'

In [ ]:
%%writefile {_trainer_module_file}

from typing import List
from absl import logging
import tensorflow as tf
from tensorflow import keras
from tensorflow_transform.tf_metadata import schema_utils

from tfx import v1 as tfx
from tfx_bsl.public import tfxio
from tensorflow_metadata.proto.v0 import schema_pb2

_FEATURE_KEYS = [
    'culmen_length_mm', 'culmen_depth_mm', 'flipper_length_mm', 'body_mass_g'
]
_LABEL_KEY = 'species'

_TRAIN_BATCH_SIZE = 20
_EVAL_BATCH_SIZE = 10

# Since we're not generating or creating a schema, we will instead create
# a feature spec.  Since there are a fairly small number of features this is
# manageable for this dataset.
_FEATURE_SPEC = {
    **{
        feature: tf.io.FixedLenFeature(shape=[1], dtype=tf.float32)
           for feature in _FEATURE_KEYS
       },
    _LABEL_KEY: tf.io.FixedLenFeature(shape=[1], dtype=tf.int64)
}


def _input_fn(file_pattern: List[str],
              data_accessor: tfx.components.DataAccessor,
              schema: schema_pb2.Schema,
              batch_size: int = 200) -> tf.data.Dataset:
  """Generates features and label for training.

  Args:
    file_pattern: List of paths or patterns of input tfrecord files.
    data_accessor: DataAccessor for converting input to RecordBatch.
    schema: schema of the input data.
    batch_size: representing the number of consecutive elements of returned
      dataset to combine in a single batch

  Returns:
    A dataset that contains (features, indices) tuple where features is a
      dictionary of Tensors, and indices is a single Tensor of label indices.
  """
  return data_accessor.tf_dataset_factory(
      file_pattern,
      tfxio.TensorFlowDatasetOptions(
          batch_size=batch_size, label_key=_LABEL_KEY),
      schema=schema).repeat()


def _build_keras_model() -> tf.keras.Model:
  """Creates a DNN Keras model for classifying penguin data.

  Returns:
    A Keras Model.
  """
  # The model below is built with Functional API, please refer to
  # https://www.tensorflow.org/guide/keras/overview for all API options.
  inputs = [keras.layers.Input(shape=(1,), name=f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  for _ in range(2):
    d = keras.layers.Dense(8, activation='relu')(d)
  outputs = keras.layers.Dense(3)(d)

  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(
      optimizer=keras.optimizers.Adam(1e-2),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[keras.metrics.SparseCategoricalAccuracy()])

  model.summary(print_fn=logging.info)
  return model


# TFX Trainer will call this function.
def run_fn(fn_args: tfx.components.FnArgs):
  """Train the model based on given args.

  Args:
    fn_args: Holds args used to train the model as name/value pairs.
  """

  # This schema is usually either an output of SchemaGen or a manually-curated
  # version provided by pipeline author. A schema can also derived from TFT
  # graph if a Transform component is used. In the case when either is missing,
  # `schema_from_feature_spec` could be used to generate schema from very simple
  # feature_spec, but the schema returned would be very primitive.
  schema = schema_utils.schema_from_feature_spec(_FEATURE_SPEC)

  train_dataset = _input_fn(
      fn_args.train_files,
      fn_args.data_accessor,
      schema,
      batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(
      fn_args.eval_files,
      fn_args.data_accessor,
      schema,
      batch_size=_EVAL_BATCH_SIZE)

  model = _build_keras_model()
  model.fit(
      train_dataset,
      steps_per_epoch=fn_args.train_steps,
      validation_data=eval_dataset,
      validation_steps=fn_args.eval_steps)

  # The result of the training should be saved in `fn_args.serving_model_dir`
  # directory.
  model.save(fn_args.serving_model_dir, save_format='tf')

Agora você concluiu todas as etapas de preparação para construir um pipeline TFX.

### Escreva uma definição de pipeline

Definimos uma função para criar um pipeline TFX. Um objeto `Pipeline` representa um pipeline do TFX que pode ser executado usando um dos sistemas de orquestração de pipeline suportados pelo TFX.


In [ ]:
def _create_pipeline(pipeline_name: str, pipeline_root: str, data_root: str,
                     module_file: str, serving_model_dir: str,
                     metadata_path: str) -> tfx.dsl.Pipeline:
  """Creates a three component penguin pipeline with TFX."""
  # Brings data into the pipeline.
  example_gen = tfx.components.CsvExampleGen(input_base=data_root)

  # Uses user-provided Python function that trains a model.
  trainer = tfx.components.Trainer(
      module_file=module_file,
      examples=example_gen.outputs['examples'],
      train_args=tfx.proto.TrainArgs(num_steps=100),
      eval_args=tfx.proto.EvalArgs(num_steps=5))

  # Pushes the model to a filesystem destination.
  pusher = tfx.components.Pusher(
      model=trainer.outputs['model'],
      push_destination=tfx.proto.PushDestination(
          filesystem=tfx.proto.PushDestination.Filesystem(
              base_directory=serving_model_dir)))

  # Following three components will be included in the pipeline.
  components = [
      example_gen,
      trainer,
      pusher,
  ]

  return tfx.dsl.Pipeline(
      pipeline_name=pipeline_name,
      pipeline_root=pipeline_root,
      metadata_connection_config=tfx.orchestration.metadata
      .sqlite_metadata_connection_config(metadata_path),
      components=components)

## Execute o pipeline

O TFX oferece suporte a múltiplos orquestradores para executar pipelines. Neste tutorial usaremos `LocalDagRunner` que está incluído no pacote TFX Python e executa pipelines em ambiente local. Freqüentemente chamamos os pipelines TFX de "DAGs", que significa gráfico acíclico direcionado.

`LocalDagRunner` fornece iterações rápidas para desenvolvimento e depuração. O TFX também oferece suporte a outros orquestradores, incluindo Kubeflow Pipelines e Apache Airflow, que são adequados para casos de uso em produção.

Consulte [TFX em pipelines Cloud AI Platform](https://www.tensorflow.org/tfx/tutorials/tfx/cloud-ai-platform-pipelines) ou o [Tutorial do TFX Airflow](https://www.tensorflow.org/tfx/tutorials/tfx/airflow_workshop) para saber mais sobre outros sistemas de orquestração.

Agora criaremos um `LocalDagRunner` e passaremos um objeto `Pipeline` criado a partir da função que já definimos.

O pipeline é executado e você poderá ver os logs do progresso do pipeline, incluindo o treinamento do modelo de ML.

In [ ]:
tfx.orchestration.LocalDagRunner().run(
  _create_pipeline(
      pipeline_name=PIPELINE_NAME,
      pipeline_root=PIPELINE_ROOT,
      data_root=DATA_ROOT,
      module_file=_trainer_module_file,
      serving_model_dir=SERVING_MODEL_DIR,
      metadata_path=METADATA_PATH))

Você deverá ver a mensagem "INFO:absl:Component Pusher is finished." no final dos logs se o pipeline for concluído com sucesso. Porque o componente `Pusher` é o último componente do pipeline.

O componente pusher envia o modelo treinado para `SERVING_MODEL_DIR` que é o diretório `serving_model/penguin-simple` se você não alterou as variáveis ​​nas etapas anteriores. Você pode ver o resultado no navegador de arquivos no painel esquerdo do Colab ou usando o seguinte comando:

In [ ]:
# List files in created model directory.
!find {SERVING_MODEL_DIR}

## Próximos passos

Você encontrará mais recursos em https://www.tensorflow.org/tfx/tutorials.

Veja [Introdução aos pipelines do TFX](https://www.tensorflow.org/tfx/guide/understanding_tfx_pipelines) para saber mais sobre vários conceitos do TFX.
